# Webscrapping des données Sentix

## Obtenir la bonne page HTML avec Selenium

Avant de commencer le webscrapping, nous devons effectuer quelques modifications dans le code HMTML de base de la pgae web. En effet, pour obtenir l'intégralité des données, la page initiale requiert d'appuyer plusieurs fois sur le bouton "Voir plus". Ainsi, pour obtenir le jeu de données entier, nous allons utiliser la librairie **selenium** qui nous permettra d'obtenir la page HTML avec toutes les données.

In [99]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [100]:
# Les deux lignes suivantes pemettent d'empêcher l'apparition de pop up (cookies, agreement...) sur la page
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.cookies": 2})

driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
driver.get('https://fr.investing.com/economic-calendar/sentix-investor-confidence-268')

# On chercher le bouton "Voir plus" pour pouvoir ensuite appuyer dessus
search = driver.find_element("id","showMoreHistory268")

# On afficher toutes les données tant que le bouton "Voir plus" est cliquable 
open=True
while len(driver.find_elements("id", "showMoreHistory268"))>0:
    try:
        show_more = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(("id","showMoreHistory268"))).click() # on attend 10 secondes avant de break en l'abscence du bouton
    except:
        print("No more data to show")
        break

webpage=driver.page_source #on stock le code html de la page où toutes le données sont chargées
driver.quit()     

C:\Users\elelo\AppData\Local\Temp/ipykernel_10420/261217928.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)


No more data to show


## From HTML to Dataframe :

Maintenant que nous avons le code HTML de toutes les données, nous allons décortiquer la page grâce à la librairie BeautifulSoup.

In [101]:
import bs4
import pandas

In [102]:
# utiliser le package BeautifulSoup qui "comprend" les balises 

page = bs4.BeautifulSoup(webpage, "lxml")

print(page)

<html class="fr js csstransitions" dir="ltr" geo="BE" lang="fr" xml:lang="fr" xmlns="http://www.w3.org/1999/xhtml" xmlns:schema="http://schema.org/"><head>
<link href="https://i-invdn-com.investing.com" rel="dns-prefetch"/>
<link href="https://a-invdn-com.investing.com" rel="dns-prefetch"/>
<link href="https://securepubads.g.doubleclick.net" rel="preconnect"/>
<link href="https://securepubads.g.doubleclick.net" rel="dns-prefetch"/>
<script async="" src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.lb.fr.vUK8YC44CcM.O/m=auth2/rt=j/sv=1/d=1/ed=1/rs=AHpOoo9V1iR9HeQfXrHB5nhWZ1ppqHB7_A/cb=gapi.loaded_0?le=scs"></script><script async="" src="https://sb.scorecardresearch.com/beacon.js"></script><script async="" src="https://cdn.taboola.com/libtrc/unip/1407007/tfa.js" type="text/javascript"></script><script async="" src="https://connect.facebook.net/en_US/fbevents.js" type="text/javascript"></script><script async="" gtm="GTM-PGT3R3D" src="https://www.googleoptimize.com/optimize.js?id=G

In [103]:
# on identifie le tableau des indices sentix : c'est le premier qui a cette classe "genTbl openTbl ecHistoryTbl"
tableau_indice = page.find('table', {'class' : 'genTbl openTbl ecHistoryTbl'})

print(tableau_indice)

<table class="genTbl openTbl ecHistoryTbl" id="eventHistoryTable268">
<thead>
<tr>
<th class="left symbol noWrap">Date de sortie</th>
<th class="left time noWrap">Heure</th>
<th class="act noWrap">Actuel</th>
<th class="for noWrap">Prév.</th>
<th class="pre noWrap">Précédent</th>
<th></th>
</tr>
</thead>
<tbody> <tr event_attr_id="268" event_timestamp="2022-11-07 09:30:00" id="historicEvent_460722">
<td class="left">07/11/2022 (Nov.)</td>
<td class="left">10:30</td>
<td class="noWrap"><span class="blackFont" title=""> </span></td>
<td class="noWrap">-35,0</td>
<td class="blackFont noWrap">-38,3</td>
<td class="icon center"><i class="diamondNewEmptyIcon"></i></td>
</tr>
<tr event_attr_id="268" event_timestamp="2022-10-10 08:30:00" id="historicEvent_458993">
<td class="left">10/10/2022 (Oct.)</td>
<td class="left">09:30</td>
<td class="noWrap"><span class="redFont" title="Inférieur aux prévisions">-38,3</span></td>
<td class="noWrap">-34,7</td>
<td class="blackFont noWrap">-31,8</td>
<td

In [104]:
table_body = tableau_indice.find('tbody')

# on recherche toutes les lignes du tableau avec la balise "tr"
rows = table_body.find_all('tr')

# on obtient une liste où chaque élément est une des lignes du tableau


In [105]:
cols = rows[1].find_all('td')
print(cols[0])
print(cols[0].text.strip())

<td class="left">10/10/2022 (Oct.)</td>
10/10/2022 (Oct.)


In [106]:
for ele in cols : 
    print(ele.text.strip())

10/10/2022 (Oct.)
09:30
-38,3
-34,7
-31,8



Voici le dictionnaire de tous les indices Sentix :

In [108]:
dico_date = dict()
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if len(cols) > 0 : 
        dico_date[cols[0]] = cols[1:]
dico_date

{'07/11/2022 (Nov.)': ['10:30', '', '-35,0', '-38,3', ''],
 '10/10/2022 (Oct.)': ['09:30', '-38,3', '-34,7', '-31,8', ''],
 '05/09/2022 (Sept.)': ['09:30', '-31,8', '-27,5', '-25,2', ''],
 '08/08/2022 (Août)': ['09:30', '-25,2', '-24,7', '-26,4', ''],
 '04/07/2022 (Juill.)': ['09:30', '-26,4', '-19,9', '-15,8', ''],
 '07/06/2022 (Juin)': ['09:30', '-15,8', '-20,0', '-22,6', ''],
 '09/05/2022  (Mai)': ['09:30', '-22,6', '-20,8', '-18,0', ''],
 '04/04/2022  (Avr)': ['09:30', '-18,0', '-9,2', '-7,0', ''],
 '07/03/2022  (Mar)': ['10:30', '-7,0', '5,3', '16,6', ''],
 '07/02/2022  (Févr.)': ['10:30', '16,6', '15,2', '14,9', ''],
 '10/01/2022  (Jan.)': ['10:30', '14,9', '12,0', '13,5', ''],
 '06/12/2021  (Déc)': ['10:30', '13,5', '15,9', '18,3', ''],
 '08/11/2021  (Nov.)': ['10:30', '18,3', '15,5', '16,9', ''],
 '04/10/2021  (Oct.)': ['09:30', '16,9', '18,6', '19,6', ''],
 '06/09/2021  (Sept.)': ['09:30', '19,6', '19,7', '22,2', ''],
 '09/08/2021  (Août)': ['09:30', '22,2', '29,0', '29,8', ''

In [109]:
data_sentix = pandas.DataFrame.from_dict(dico_date,orient='index')
data_sentix.head()

,0,1,2,3,4
07/11/2022 (Nov.),10:30,,"-35,0","-38,3",
10/10/2022 (Oct.),09:30,"-38,3","-34,7","-31,8",
05/09/2022 (Sept.),09:30,"-31,8","-27,5","-25,2",
08/08/2022 (Août),09:30,"-25,2","-24,7","-26,4",
04/07/2022 (Juill.),09:30,"-26,4","-19,9","-15,8",


In [110]:
table_head=page.find("thead")
col_name_head=[]
for x in table_head.find_all("tr"):
    for y in x.find_all('th'):
        col_name_head.append(y.text)
col_name_head

['Date de sortie', 'Heure', 'Actuel', 'Prév.', 'Précédent', '']

In [111]:
data_sentix.columns=col_name_head[1:]

In [112]:
data_sentix

,Heure,Actuel,Prév.,Précédent,
07/11/2022 (Nov.),10:30,,"-35,0","-38,3",
10/10/2022 (Oct.),09:30,"-38,3","-34,7","-31,8",
05/09/2022 (Sept.),09:30,"-31,8","-27,5","-25,2",
08/08/2022 (Août),09:30,"-25,2","-24,7","-26,4",
04/07/2022 (Juill.),09:30,"-26,4","-19,9","-15,8",
...,...,...,...,...,...
01/04/2003 (Mar),10:00,"-19,40",,"-22,80",
01/03/2003 (Févr.),10:00,"-22,80",,"-21,80",
01/02/2003 (Jan.),10:00,"-21,80",,"-8,50",
01/10/2002 (Sept.),10:00,"-8,50",,"13,00",
